In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

 99% 1.31G/1.32G [00:08<00:00, 147MB/s]
100% 1.32G/1.32G [00:08<00:00, 159MB/s]


In [ ]:
!unzip -qq gender-recognition-200k-images-celeba.zip

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from config import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dataset

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1/.255,
    horizontal_flip=True,
    # zoom_range=0.3,
    # rotation_range=30
)

train_data = data_generator.flow_from_directory(
    "/content/Dataset/Train",
    target_size = (width, height),
    batch_size = batch_size,
    # class_mode = 'binary',
    class_mode = 'categorical',
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    "/content/Dataset/Validation",
    target_size = (width, height),
    batch_size = batch_size,
    # class_mode = 'binary',
    class_mode = 'categorical',
    shuffle = True,
)

print(np.bincount(train_data.labels))
print(np.bincount(val_data.labels))

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
[92845 67155]
[13778  8820]


In [ ]:
model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
])
# model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                             metrics=['accuracy'])
# model.compile(loss=tf.keras.losses.binary_crossentropy,
#               optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
#                              metrics=['accuracy'])

In [ ]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=epochs,
          class_weight={0: 1, 1: 1.4}
          )

Epoch 1/15
3333/3333 [==============================] - 453s 136ms/step - loss: 0.1649 - accuracy: 0.9438 - val_loss: 0.0849 - val_accuracy: 0.9667
Epoch 2/15
3333/3333 [==============================] - 444s 133ms/step - loss: 0.0855 - accuracy: 0.9720 - val_loss: 0.0725 - val_accuracy: 0.9714
Epoch 3/15
3333/3333 [==============================] - 444s 133ms/step - loss: 0.0691 - accuracy: 0.9780 - val_loss: 0.0782 - val_accuracy: 0.9705
Epoch 4/15
3333/3333 [==============================] - 445s 133ms/step - loss: 0.0567 - accuracy: 0.9823 - val_loss: 0.0621 - val_accuracy: 0.9761
Epoch 5/15
3333/3333 [==============================] - 444s 133ms/step - loss: 0.0473 - accuracy: 0.9853 - val_loss: 0.0810 - val_accuracy: 0.9703
Epoch 6/15
3333/3333 [==============================] - 444s 133ms/step - loss: 0.0389 - accuracy: 0.9883 - val_loss: 0.0692 - val_accuracy: 0.9752
Epoch 7/15
3333/3333 [==============================] - 445s 133ms/step - loss: 0.0334 - accuracy: 0.9900 - val_

In [ ]:
# %cd /content/drive/MyDrive/DeepLearningTasks/ GenderClassification
model.save("/content/drive/MyDrive/DeepLearningTasks/ GenderClassification/model4.h5")

In [ ]:
test_data_path = '/content/Dataset/Test'

data_generator = ImageDataGenerator(rescale = 1/.255)

test_data = data_generator.flow_from_directory(
    test_data_path,
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical')

model.evaluate(test_data)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[6821 4721]
 [4880 3579]]
Classification Report
              precision    recall  f1-score   support

      Female       0.58      0.59      0.59     11542
        Male       0.43      0.42      0.43      8459

    accuracy                           0.52     20001
   macro avg       0.51      0.51      0.51     20001
weighted avg       0.52      0.52      0.52     20001

